In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 4
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.21 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        
        #------------------------------------------------
        #需要調整的地方
        #------------------------------------------------
        #--要用幾秒訓練 data_1S/data_3S/data_1S
        #--學習率
        #--epoch
        if(TL==0):
            trainData = data_5S
        elif(TL==1):
            trainData = data_3S
        else:
            trainData = data_1S
        #------------------------------------------------
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型
        model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold)
    
        #測試準確度
        test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
        test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
        test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1


C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torchvision\ops\misc.py:120: UserWarning: Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead.
  warnings.warn(
Downloading: "https://github.com/fschmid56/EfficientAT/releases/download/v0.0.1/mn40_as_ext_mAP_487.pt" to resources\mn40_as_ext_mAP_487.pt
100.0%


Number of classes defined: 10, but try to load pre-trained layer with logits: 527
Dropping last layer.
Error(s) in loading state_dict for MobileNetV3:
	Missing key(s) in state_dict: "classifier.5.weight", "classifier.5.bias". 
Loading weights pre-trained weights in a non-strict manner.
MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=Fa

----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------


C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torch\functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\SpectralOps.cpp:867.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


...

Train Epoch: 1/50 Train_Loss: 0.34765610098838806 Train_acc: 90.437500%
Train Epoch: 2/50 Train_Loss: 0.2521321773529053 Train_acc: 90.250000%
Train Epoch: 3/50 Train_Loss: 0.22316981852054596 Train_acc: 90.937500%
Train Epoch: 4/50 Train_Loss: 0.18800115585327148 Train_acc: 90.500000%
Train Epoch: 5/50 Train_Loss: 0.4689539074897766 Train_acc: 90.437500%
Train Epoch: 6/50 Train_Loss: 0.1553303450345993 Train_acc: 90.500000%
Train Epoch: 7/50 Train_Loss: 0.13125617802143097 Train_acc: 91.000000%
Train Epoch: 8/50 Train_Loss: 0.18946826457977295 Train_acc: 91.500000%
Train Epoch: 9/50 Train_Loss: 0.21808768808841705 Train_acc: 92.687500%
Train Epoch: 10/50 Train_Loss: 0.28803879022598267 Train_acc: 92.562500%
Train Epoch: 11/50 Train_Loss: 0.1577751189470291 Train_acc: 93.312500%
Train Epoch: 12/50 Train_Loss: 0.11018099635839462 Train_acc: 92.937500%
Train Epoch: 13/50 Train_Loss: 0.17775830626487732 Train_acc: 93.500000%
Train Epoch: 14/50 Train_Loss: 0.14917325973510742 Train_acc: 93

----------------------------------------------------------------------------------------------------------------
fold:  4 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/50 Train_Loss: 2.301861524581909 Train_acc: 18.125000%
Train Epoch: 2/50 Train_Loss: 2.240999937057495 Train_acc: 16.250000%
Train Epoch: 3/50 Train_Loss: 2.2429723739624023 Train_acc: 15.937500%
Train Epoch: 4/50 Train_Loss: 2.259901285171509 Train_acc: 16.250000%
Train Epoch: 5/50 Train_Loss: 2.242868185043335 Train_acc: 15.937500%
Train Epoch: 6/50 Train_Loss: 2.273254632949829 Train_acc: 20.312500%
Train Epoch: 7/50 Train_Loss: 2.151414155960083 Train_acc: 25.312500%
Train Epoch: 8/50 Train_Loss: 2.066800117492676 Train_acc: 38.125000%
Train Epoch: 9/50 Train_Loss: 2.019554615020752 Train_acc: 49.062500%
Train Epoch: 10/50 Train_Loss: 1.9285186529159546 Train_acc: 61.875000%
Train Epoch: 11/50 Train_Loss: 1.8013397455215454 Train

Train Epoch: 1/50 Train_Loss: 0.2715676724910736 Train_acc: 89.875000%
Train Epoch: 2/50 Train_Loss: 0.34444159269332886 Train_acc: 89.687500%
Train Epoch: 3/50 Train_Loss: 0.3395555019378662 Train_acc: 89.875000%
Train Epoch: 4/50 Train_Loss: 0.23496004939079285 Train_acc: 89.937500%
Train Epoch: 5/50 Train_Loss: 0.23530125617980957 Train_acc: 89.875000%
Train Epoch: 6/50 Train_Loss: 0.32889485359191895 Train_acc: 90.750000%
Train Epoch: 7/50 Train_Loss: 0.2597123384475708 Train_acc: 91.312500%
Train Epoch: 8/50 Train_Loss: 0.1942499279975891 Train_acc: 91.062500%
Train Epoch: 9/50 Train_Loss: 0.17897659540176392 Train_acc: 91.562500%
Train Epoch: 10/50 Train_Loss: 0.2331223487854004 Train_acc: 92.125000%
Train Epoch: 11/50 Train_Loss: 0.20636603236198425 Train_acc: 92.812500%
Train Epoch: 12/50 Train_Loss: 0.07269380241632462 Train_acc: 93.187500%
Train Epoch: 13/50 Train_Loss: 0.2917409837245941 Train_acc: 92.875000%
Train Epoch: 14/50 Train_Loss: 0.1443449705839157 Train_acc: 92.62

----------------------------------------------------------------------------------------------------------------
fold:  5 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/50 Train_Loss: 2.3037660121917725 Train_acc: 8.125000%
Train Epoch: 2/50 Train_Loss: 2.32016658782959 Train_acc: 7.500000%
Train Epoch: 3/50 Train_Loss: 2.3179192543029785 Train_acc: 6.875000%
Train Epoch: 4/50 Train_Loss: 2.315812826156616 Train_acc: 9.062500%
Train Epoch: 5/50 Train_Loss: 2.279296875 Train_acc: 10.000000%
Train Epoch: 6/50 Train_Loss: 2.2731680870056152 Train_acc: 14.375000%
Train Epoch: 7/50 Train_Loss: 2.195732593536377 Train_acc: 19.062500%
Train Epoch: 8/50 Train_Loss: 2.1300525665283203 Train_acc: 25.625000%
Train Epoch: 9/50 Train_Loss: 2.048220157623291 Train_acc: 39.687500%
Train Epoch: 10/50 Train_Loss: 1.9330581426620483 Train_acc: 56.250000%
Train Epoch: 11/50 Train_Loss: 1.8220206499099731 Train_acc: 74

Train Epoch: 1/50 Train_Loss: 0.328773558139801 Train_acc: 89.312500%
Train Epoch: 2/50 Train_Loss: 0.33733516931533813 Train_acc: 89.812500%
Train Epoch: 3/50 Train_Loss: 0.34543508291244507 Train_acc: 89.937500%
Train Epoch: 4/50 Train_Loss: 0.31676363945007324 Train_acc: 89.375000%
Train Epoch: 5/50 Train_Loss: 0.4414198398590088 Train_acc: 90.250000%
Train Epoch: 6/50 Train_Loss: 0.3804963529109955 Train_acc: 90.125000%
Train Epoch: 7/50 Train_Loss: 0.24029694497585297 Train_acc: 89.562500%
Train Epoch: 8/50 Train_Loss: 0.25801417231559753 Train_acc: 91.687500%
Train Epoch: 9/50 Train_Loss: 0.39678555727005005 Train_acc: 90.937500%
Train Epoch: 10/50 Train_Loss: 0.13033653795719147 Train_acc: 91.562500%
Train Epoch: 11/50 Train_Loss: 0.10537021607160568 Train_acc: 91.812500%
Train Epoch: 12/50 Train_Loss: 0.25843408703804016 Train_acc: 91.875000%
Train Epoch: 13/50 Train_Loss: 0.10926929861307144 Train_acc: 92.750000%
Train Epoch: 14/50 Train_Loss: 0.09097252041101456 Train_acc: 93

In [5]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()